In [1]:
library(tidyverse)
library(sloop)

-- Attaching packages ----------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.2     v purrr   0.3.4
v tibble  3.0.4     v dplyr   1.0.2
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.0

-- Conflicts -------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



# Goals

- Why use s3 class?
- What is a S3 class?
- What is a **generic**?
- What is a **method**?
- What is **method dispatch**?
- How you can make an object be an instance of a class?

# 1. Basics

S3 consists of 3 components:
- class atribute
- generics
- methods


S3 classes are implemented using attributes. An S3 object is an base type with at least a `class` attribute. For example, a data frame has base type is list and clas is data.frame

In [61]:
typeof(mtcars)
class(mtcars)

[1] "list"

[1] "data.frame"

A **generic** is a function that behaves differently for different class. For example, `print` function is a **generic**

> To check if a function is **generic**, use `sloop::ftype` and look for generics. Or you can read the source and check if that functions has **`UseMethod`**

In [62]:
# print is a generic
sloop::ftype(print)

[1] "S3"      "generic"

In [63]:
# source code
print

function (x, ...) 
UseMethod("print")
<bytecode: 0x5563d9025bd0>
<environment: namespace:base>

A **method** is a specific function implementation for a S3 class. A **generic** find a **method** by using **method dispatch**

In [12]:
# this is not a generic, but a method
sloop::ftype(print.data.frame)

[1] "S3"     "method"

In [14]:
# method dispatch for mtcars (a data.frame object)
sloop::s3_dispatch(print(mtcars))

=> print.data.frame
 * print.default

If a generic does not have a method for a specific class, it will use **`generic.default`**

In [67]:
my_instance <- structure(1:10, class = 'grade')

print(my_instance)

 [1]  1  2  3  4  5  6  7  8  9 10
attr(,"class")
[1] "grade"


In [68]:
# equivalent
print.default(my_instance)

 [1]  1  2  3  4  5  6  7  8  9 10
attr(,"class")
[1] "grade"


# 2. Classes

>To make an object be an instance of a class, just assign the class attribute to it

In [66]:
tanks <- c('T14 Armata', 'MZ 51', 'Challenger')

In [19]:
# using structure
structure(tanks, class = 'tanks')

[1] "T14 Armata" "MZ 51"      "Challenger"
attr(,"class")
[1] "tanks"

In [20]:
# or modify in-place
class(tanks) <- 'tanks'
tanks

[1] "T14 Armata" "MZ 51"      "Challenger"
attr(,"class")
[1] "tanks"

>Determine the class of a S3 object:

In [21]:
class(tanks)

[1] "tanks"

>Check if an object is an instance of a class

In [22]:
inherits(tanks, 'tanks')

[1] TRUE

---
I recommend that you usually provide three functions:
- A **constructor** that has the form `new_myclass`,  that efficiently creates new objects with the correct structure.
- A **validator** that has the form `validate_myclass`, that performs more computationally expensive checks to ensure that the object has correct values.
- A **helper** that hass the form `myclass`, that provides a convenient way for others to create objects of your class.

> Build a polynomial class having positive coefficients

In [27]:
# constructor a poly
new_polynomial <- function(x = double()) {
    # all coefficients must be a number
    stopifnot(is.double(x))
    structure(x, class = 'polynomial')
}

In [38]:
# validator
validate_polynomial <- function(poly) {
    # check if coefficients are positive
    coefficients <- unclass(poly)
    if(any(coefficients <= 0))
        stop("`x` must be positive", call. = FALSE)
    poly
}

In [39]:
# helper
polynomial <- function(x) {
    validate_polynomial(new_polynomial(x))
}

In [44]:
# method for class polynomial from generic print
print.polynomial <- function(poly) {
    coefs <- unclass(poly)
    nums <- length(coefs)
    pow <- 1:nums
    x <- str_c('x^', nums - pow)
    str_c(coefs, x, sep = '*', collapse = '+')
    
}

In [45]:
equation <- polynomial(c(3, 5, 1))

print(equation)

[1] "3*x^2+5*x^1+1*x^0"

In [46]:
# invalid argument
try(polynomial(c(-5, 2, 8)))

Error : `x` must be positive


# 3.Generics and methods

The job of an S3 generic is to perform method dispatch, i.e. find the specific implementation for a class. Method dispatch is performed by **`UseMethod()`**, which every generic calls. **`UseMethod()`** takes two arguments: the name of the generic function (required), and the argument to use for method dispatch (optional). If you omit the second argument, it will dispatch based on the first argument, which is almost always what is desired.

In [49]:
mean

function (x, ...) 
UseMethod("mean")
<bytecode: 0x5563d702c7b8>
<environment: namespace:base>

In [50]:
# create your own generic function
my_generic_func <- function(x, ...) {
    UseMethod("my_generic_func")
}

see methods for a generic

In [53]:
methods('mean')

[1] mean.Date        mean.POSIXct     mean.POSIXlt     mean.default    
[5] mean.difftime    mean.quosure*    mean.vctrs_vctr*
see '?methods' for accessing help and source code

In [59]:
# equivalent
sloop::s3_methods_generic('mean')

generic,class,visible,source
<chr>,<chr>,<lgl>,<chr>
mean,Date,TRUE,base
mean,POSIXct,TRUE,base
mean,POSIXlt,TRUE,base
mean,default,TRUE,base
mean,difftime,TRUE,base
mean,quosure,FALSE,registered S3method
mean,vctrs_vctr,FALSE,registered S3method


See generics for a class

In [60]:
sloop::s3_methods_class('table')

generic,class,visible,source
<chr>,<chr>,<lgl>,<chr>
Axis,table,FALSE,registered S3method
[,table,TRUE,base
aperm,table,TRUE,base
as.data.frame,table,TRUE,base
as_tibble,table,FALSE,registered S3method
lines,table,FALSE,registered S3method
plot,table,FALSE,registered S3method
points,table,FALSE,registered S3method
print,table,TRUE,base


In [57]:
# equivalent
methods(class = 'table')

 [1] Axis          [             aperm         as.data.frame as_tibble    
 [6] coerce        initialize    lines         plot          points       
[11] print         show          slotsFromS3   summary       tail         
see '?methods' for accessing help and source code

# 4.Inheritance

3 ideas of Inheritance:
- The class can be a character vector

In [2]:
class(diamonds)

[1] "tbl_df"     "tbl"        "data.frame"

In [3]:
class(ordered(letters))

[1] "ordered" "factor"

`ordered` is a **subclass** of `factor`. `factor` is a **superclass** of `ordered`

- If a method is not found for the class of the first element of the vector, it looks for a method in the second class (and so on)

In [4]:
s3_dispatch(print(diamonds))

   print.tbl_df
=> print.tbl
 * print.data.frame
 * print.default

- A method can delegate work by using **`NextMethod()`**

In [8]:
# subsetting still preserve class
s3_dispatch(diamonds[, 1:3])

=> [.tbl_df
   [.tbl
 * [.data.frame
   [.default
 * [ (internal)

## 4.1 `NextMethod`

Let's create a class that receives a double vector an returns a characters vector

In [10]:
new_repetition <- function(freq = double()) {
    stopifnot(is.double(freq))
    structure(freq, class = "repetition")
}

print.repetition <- function(freq) {
    print(str_dup("x", freq))
    invisible(freq)
}

x <- new_repetition(c(2, 4, 3))

print(x)

[1] "xx"   "xxxx" "xxx" 


We expect `x[1]` returns "xx", `x[2]` returns "xxxx" and so on ....

In [12]:
x[1]

[1] 2

In [13]:
x[2]

[1] 4

Well, it does not work. Subset must return the same class, but in this case, it does not

In [14]:
class(x[1])

[1] "numeric"

So we have to implement a subsetting function for class `repetition`
The below approach won't work because we will have infinite loop

```r
`[.repetition` <- function(x, i) {
    new_repetition(x[i])
    # equivalent
    # new_repetition(`[.repetition`(x, i))
}
```

One approach is to unclass `x`

In [21]:
`[.repetition` <- function(x, i) {
    x <- unclass(x)
    # now x is a numeric class so we won't have infinite loop
    new_repetition(x[i])
}

print(x[1])

class(x[1])

[1] "xx"


[1] "repetition"

This is exactly what **`NextMethod()`** do. We can think of **`NextMethod()`** will use the subset method `[` of parent class of `x`, which in this case, is a numeric class.

In [22]:
`[.repetition` <- function(x, i) {
    new_repetition(NextMethod())
}

print(x[1])

class(x[1])

[1] "xx"


[1] "repetition"

**`NextMethod()`**
doesn’t actually work with the class attribute of the object, but instead uses a special global variable (`.Class`) to keep track of which method to call next.

In [62]:
generic2 <- function(x) UseMethod("generic2")
generic2.a1 <- function(x) "a1"
generic2.a2 <- function(x) "a2"
generic2.b <- function(x) {
  class(x) <- "a1"
  print(.Class)
  NextMethod()
}

generic2(structure(list(), class = c("b", "a2")))

[1] "b"  "a2"


[1] "a2"

What is happening here?  
We pass an object of class "b" and "a2" to `generic2()`, which prompts R to look for a method `generic2.b`  
Then this function class the class of this object to "a1" and calls **`NextMethod()`**, look at the result of `print(.Class)`, we know that `NextMethod()` will call `generic2.a2`, NOT `generic2.a1`

What would have if we remove the line `class(x) <- "a1"?

In [63]:
generic2 <- function(x) UseMethod("generic2")
generic2.a1 <- function(x) "a1"
generic2.a2 <- function(x) "a2"
generic2.b <- function(x) {
  print(.Class)
  NextMethod()
}

generic2(structure(list(), class = c("b", "a2")))

[1] "b"  "a2"


[1] "a2"

Nothing change. So we can confirm that **`NextMethod`** does not work with the attribute `class` of an object. It means it does not care about the value of the `class` attribute of that object. It use a special global variable `.Class` to keep track of what method should be called next. In this case, no matter what value you change to `class` attribute to, NextMethod() will always find next method in the order "b", "a2" for the above implementation

## 4.2 Allowing subclassing

Let's create a class that inherits from class `secret`. To allow subclasses, the parent constructor needs to have `...` and `class` arguments:

In [44]:
# redefine the constructor for `repetition` class
new_repetition <- function(x, ..., class = character()) {
    stopifnot(is.double(x))
    structure(x, ..., class = c(class, 'repetition'))
}
# methods of class `repetition`
print.repetition <- function(x) {
    print(str_dup("x", x))
}
`[.repetition` <- function(x, ...) {
    # use ... to allow subset 1 element or slice, ... etc
    new_repetition(NextMethod())
}
# constructor for subclass `complexrep` inherits from `repetition`
new_complexrep <- function(x) {
    new_repetition(x, class = 'complexrep')
}
# overwrite the print method of parent class `repetition`
print.complexrep <- function(x) {
    print(str_dup('xXx', x))
    invisible(x)
}

y <- new_complexrep(c(4, 2, 3))

print(y)

[1] "xXxxXxxXxxXx" "xXxxXx"       "xXxxXxxXx"   


In [45]:
s3_dispatch(print(y))

=> print.complexrep
 * print.repetition
 * print.default

Because `complexrep` inherits from `repetition`, so it will have to subset method of `repetition`

In [46]:
s3_dispatch(y[1])

   [.complexrep
=> [.repetition
   [.default
-> [ (internal)

```r
`[.repetition` <- function(x, i) {
    new_repetition(NextMethod())
}
```

In [47]:
# using `[.pepetition`, a method of parent class
y[1]
class(y[1])

[1] "xxxx"

[1] "repetition"

we want `[.repetition` return the same class of `x`, even if `x` is a subclass

> solution: Can't be solved by using base R. Use **`vctrs::vec_restore()`**

Typically **`vec_restore()`** methods are quite simple: you just call the constructor with appropriate arguments:

In [49]:
vec_restore.repetition <- function(x, ...) new_repetition(x)
vec_restore.complexrep <- function(x, ...) new_complexrep(x)

now use **`vec_restore`** in `[.repetition` method:

In [56]:
`[.repetition` <- function(x, ...) {
    # This must return a new instance that having the same class(es) of `x`
    vctrs::vec_restore(NextMethod(), x)
}

In [57]:
x[1:2]
class(x[1:2])

[1] "xx"   "xxxx"

[1] "repetition"

In [58]:
# classes of y is preserved now
y[1:2]
class(y[1:2])

[1] "xXxxXxxXxxXx" "xXxxXx"      

[1] "complexrep" "repetition"

# 5. Dispatch details

## 5.1. S3 vs base type

What happens when you call an S3 generic with a base object, i.e. an object with no class:

In [64]:
v <- matrix(1:5)

attr(v, 'class')

NULL

Note that `class(v)` does not equivalent `attr(v, "class")`

In [67]:
class(v)

[1] "matrix" "array"

dispatch occurs on **implicit class**, there is no base function that will compute the implicit class, but we can use **`sloop::s3_class()`**

In [65]:
s3_class(v)

[1] "matrix"  "integer" "numeric"

In [66]:
s3_dispatch(print(v))

   print.matrix
   print.integer
   print.numeric
=> print.default

This means that the **`class()`** of an object does not uniquely determine its dispatched  
As far as I understand, It is determined by `attr(obj, "class")`, and if this is null, it is determined by the **implicit class**

In [70]:
vals <- 1:3
attr(vals, 'class')
s3_class(vals)
s3_dispatch(print(vals))

NULL

[1] "integer" "numeric"

   print.integer
   print.numeric
=> print.default

In [72]:
vals_2 <- structure(vals, class = 'integer')
attr(vals_2, "class")
s3_class(vals_2)
s3_dispatch(print(vals_2))

[1] "integer"

[1] "integer"

   print.integer
=> print.default